#### Import Libraries

In [229]:
import pandas as pd
from pulp import LpProblem, LpMaximize, LpVariable, lpSum
from datetime import datetime

#### Bess Optimizer Class

In [230]:
class Bess_Optimizer:
    def __init__(self, case = '',power_capacity = 100, energy_capacity = 200, efficiency = 0.9):
        self.case = case
        self.power_capacity = power_capacity
        self.energy_capacity = energy_capacity
        self.energy_prices = None
        self.reg_up_prices = None
        self.reg_down_prices = None
        self.efficiency = efficiency
        self.total_profit = None


    def load_prices(self, energy_price_file):
        '''
        load energy prices from energy_price_file
        ''' 
        self.energy_prices = pd.read_csv(energy_price_file)
        self.energy_prices['Operating Day'] = pd.to_datetime(self.energy_prices['Operating Day'])
        self.energy_prices['Date'] = self.energy_prices['Operating Day'] + pd.to_timedelta(self.energy_prices['Operating Hour'] - 1, unit='h')
        self.energy_prices['Date'] = pd.to_datetime(self.energy_prices['Date'])
        self.energy_prices['Date'] = self.energy_prices['Date'].dt.strftime("%Y_%m_%d_%H")
        

    def load_regulation(self, regulation_price_file):
        '''
        load regulation prices from regulation_price_file
        '''
        self.reg_prices = pd.read_csv(regulation_price_file)


    def optimize_period(self, start_day = None, end_day = None, initial_charge = 0):
        '''
        determine the optimal schedule for battery over the period from start_day to end_day (not included)
        includes the option to set the initial charge of the battery
        '''
        self.optimizer = LpProblem('Bess-Fluence', LpMaximize)                                            
        
        start_day = pd.to_datetime(start_day)
        end_day = pd.to_datetime(end_day)
        period = self.energy_prices[(self.energy_prices['Operating Day'] >= start_day) & (self.energy_prices['Operating Day'] <= end_day)]['Date'].tolist()
        days_list = list(self.energy_prices[(self.energy_prices['Operating Day'] >= start_day) & (self.energy_prices['Operating Day'] <= end_day)]['Operating Day'].unique())
        days_list = [day.strftime("%Y_%m_%d") for day in days_list]
        
        #Decision Variables
        
        # power generated by the battery hourly
        self.gen_hourly = LpVariable.dicts(                                                                 
                                    name = 'gen_hourly', 
                                    indices= period,
                                    lowBound = 0, 
                                    upBound = self.power_capacity
                                    )    

        # power charged by the battery hourly
        self.charge_hourly = LpVariable.dicts(                                                                 
                                    name = 'charge_hourly', 
                                    indices= period,
                                    lowBound = 0, 
                                    upBound = self.power_capacity
                                    )     
        
        # regulation up hourly
        self.reg_up_power = LpVariable.dicts(
                                        name = 'reg_up_hourly',
                                        indices= period,
                                        lowBound = 0,
                                        upBound = self.power_capacity
                                        )

        # regulation down hourly                        
        self.reg_down_power = LpVariable.dicts(
                                        name = 'reg_down_hourly',
                                        indices= period,
                                        lowBound = 0,
                                        upBound = self.power_capacity
                                        )

        # total charge hourly
        self.total_charge = LpVariable.dicts(
                                        name = 'total_charge',
                                        indices= period,
                                        lowBound = 0,
                                        upBound = self.energy_capacity
                                        )

        # objective function
        self.optimizer += lpSum(
                                self.energy_prices[self.energy_prices['Date']==hour]['Price'].iloc[0] * self.gen_hourly[hour] -
                                self.energy_prices[self.energy_prices['Date']==hour]['Price'].iloc[0] * self.charge_hourly[hour] 
                                for hour in period
                                )
        
        # iterate over the days in the period
        for day in days_list:
            hour_day = [hour for hour in period if day in hour]
            
            # only 1 cycle of charge per day
            self.optimizer += lpSum(
                                self.gen_hourly[hour] 
                                for hour in hour_day
                                ) <= self.energy_capacity * (1/self.efficiency)
        
            # only 1 cycle of charge per day
            self.optimizer += lpSum(
                                    self.charge_hourly[hour] 
                                    for hour in hour_day
                                    ) <= self.energy_capacity * (1/self.efficiency)
        

        self.optimizer += self.total_charge[period[0]] == initial_charge
        
        # temporary dependency on charginf for all hours
        for index in range(len(period)):
            if index > 0:
                past_hour = period[index-1]
                hour = period[index]
                self.optimizer += self.total_charge[hour] == self.total_charge[past_hour] + self.charge_hourly[past_hour] * self.efficiency - self.gen_hourly[past_hour] *(1/self.efficiency)
                self.optimizer += self.gen_hourly[hour] <= self.total_charge[hour] 
        
        self.optimizer.solve()
        self.total_profit = self.optimizer.objective.value()


    def get_results(self):
        '''
        print the results of the optimization and the values of total profit
        '''
        vars = ['gen_hourly', 'charge_hourly', 'reg_up_hourly', 'reg_down_hourly', 'total_charge']
        results_dict = {}

        for var in vars:
            results_dict[var] = {}
            for v in self.optimizer.variables():
                if v.name.startswith(var):
                    day_hour = v.name.replace(var + '_', '')
                    day_hour = day_hour.replace('_', '-')
                    day_hour = datetime.strptime(day_hour, '%Y-%m-%d-%H')
                    results_dict[var][day_hour] = v.varValue
        results_ds = pd.DataFrame.from_dict(results_dict)
        results_ds.to_csv(f'results-{self.case}.csv')
    
    
    def visualize(self):
        '''
        visualize the results of the optimization
        '''
        pass

     

            



In [231]:
optimizer = Bess_Optimizer(case = 'case1', power_capacity = 100, energy_capacity = 200, efficiency = 0.9)
optimizer.load_prices(energy_price_file ='data/energy_prices.csv')
optimizer.load_regulation(regulation_price_file = 'data/regulation_prices.csv')
optimizer.optimize_period(start_day = '1/1/2023', end_day = '2/1/2023', initial_charge = 0)
print(f'{optimizer.total_profit = }')
optimizer.get_results()

C:\Users\i34005\AppData\Local\Temp\ipykernel_14912\1976011167.py:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  self.energy_prices['Operating Day'] = pd.to_datetime(self.energy_prices['Operating Day'])


optimizer.total_profit = 310089.770681334
